In [1]:
import requests
from typing import Optional, Dict, Any, Union

class BaseAPI:
    """BaseAPI serves as a wrapper for basic REST API operations with a given base URL.
    
    Attributes:
        base_url (str): The base URL for the API endpoints.
        headers (Dict[str, str]): Default headers to include in all requests.
    """

    def __init__(self, base_url: str = "http://localhost:8001/v1"):
        """Initialize the BaseAPI with a base URL and default headers.

        Args:
            base_url (str): The base URL for the API endpoints. Defaults to a sample ngrok URL.
        """
        self.base_url: str = base_url
        self.headers: Dict[str, str] = {"Accept": "application/json"}

    def _post(self, endpoint: str, data: Optional[Dict[str, Any]] = None, files: Optional[Dict[str, Any]] = None):# -> Optional[Dict[str, Any]]:
        """Sends a POST request to a specific endpoint.

        Args:
            endpoint (str): The API endpoint to post data to.
            data (Optional[Dict[str, Any]]): JSON data to send in the request. Defaults to None.
            files (Optional[Dict[str, Any]]): Files to upload. Defaults to None.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        url: str = f"{self.base_url}/{endpoint}"
        headers: Dict[str, str] = self.headers
        try:
            if files:
                response = requests.post(url, files=files, headers=headers)
            else:
                response = requests.post(url, json=data, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
        return None

    def _get(self, endpoint: str) -> Optional[Dict[str, Any]]:
        """Performs a GET request to a specified endpoint.

        Args:
            endpoint (str): The API endpoint to retrieve data from.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        url: str = f"{self.base_url}/{endpoint}"
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
        return None


In [2]:
from enum import Enum, auto
from typing import List, Dict, Optional, Any

class Role(Enum):
    USER = auto()
    SYSTEM = auto()
    ASSISTANT = auto()

    def __str__(self):
        return self.name.lower()

class ChatCompletionAPI(BaseAPI):
    """
    A specialized API client for handling chat completions with conversation history management,
    utilizing an enum for role management.
    """

    def __init__(self, base_url: str = "http://localhost:8001/v1"):
        super().__init__(base_url)
        self.conversation_history: List[Dict[str, Any]] = []


    def post_completions(self, messages: List[Dict[str, Any]], role: Optional[Role] = None, stream: bool = True, context_filter: Optional[str] = None, use_context: bool = True, include_source: bool = False) -> Optional[Dict[str, Any]]:
        """
        Post messages to the chat completions endpoint with optional parameters for role, streaming, and context configuration.

        Args:
            messages: A list of message dictionaries with "content" and optionally "role" keys.
            role: An optional role for all messages, overriding individual message roles if provided. Defaults to None.
            stream: Whether to enable or disable streaming responses. Defaults to True.
            context_filter: Optional filter to apply to the context, affecting which parts are used. Defaults to None.
            use_context: Whether to use the existing context for generating completions. Defaults to True.
            include_source: Whether to include the source information in the response. Defaults to False.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        endpoint = "completions"
        payload = {
            "prompt": messages,
            "stream": stream,
            "context_filter": context_filter,
            "use_context": use_context,
            "include_source": include_source
        }
        
        # Add role to payload if it is specified
        if role is not None:
            payload["role"] = str(role)

        response = self._post(endpoint, payload)
        return response

    def post_chat_completions(self, message: str, role: Role = Role.USER, is_init: bool = False, stream: bool = False) -> Optional[Dict[str, Any]]:
        """
        Post a single message to the chat completions endpoint, managing conversation history.

        Args:
            message: The message content to send.
            role: The role of the message sender, represented as an Enum. Defaults to Role.USER.
            is_init: Whether to initialize (reset) the conversation history. Defaults to False.
            stream: Whether to enable streaming for this message. Defaults to False.

        Returns:
            The JSON response from the API or None in case of an error.
        """
        endpoint = "chat/completions"

        if is_init:
            self.conversation_history = []
            if not message:  # If initializing without a new message, return here.
                return None
            else:
                self.conversation_history.append({"content": message, "role": str(role)})
            return None

        else:
            # Add the new message with its role to the conversation history
            self.conversation_history.append({"content": message, "role": str(role)})

        payload = {
            "messages": self.conversation_history,
            "stream": stream
        }

        response = self._post(endpoint, payload)
        return response


In [3]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)

In [4]:
class UtilAPI(BaseAPI):
    def get_embeddings(self, input_text):
        """Post data to the embeddings endpoint."""
        endpoint = "embeddings"
        payload = {"input": input_text}
        return self._post(endpoint, data=payload)

    def check_health(self):
        """Get the health status of the API."""
        endpoint = "health"
        return self._get(endpoint)
    
    def post_chunks(self, text):
        """Post chunks of text."""
        endpoint = "chunks"
        payload = {"text": text}
        return self._post(endpoint, data=payload)

## Chatting

In [5]:
chat_api = ChatCompletionAPI(base_url='https://0e80-161-28-242-150.ngrok-free.app/v1')

In [6]:
# Example usage of sending a single message
messages = "What is cs 2450 prerequisit?"

response = chat_api.post_completions(messages, stream=False)

message = response['choices'][0]['message']['content']
message

' CS 2450 has two prerequisites: CS 2300 and CS 2420. Therefore, students are required to have already taken those courses before enrolling in CS 2450.'

In [7]:
# Example usage of sending a single message with role management
messages = "What is cs 2450 prerequisit?"
response = chat_api.post_completions(messages, role=Role.USER, stream=False)

message = response['choices'][0]['message']['content']
message

' The prerequisites for CS 2450 (Software Engineering) are CS 2300 and CS 2420.'

In [8]:
# Example: Chatting with continuous messaging, check the implementation how the conversation history is managed with role management
stream_response = chat_api.post_chat_completions("You are UVU Advisor", role=Role.SYSTEM, is_init=True)
stream_response = chat_api.post_chat_completions("What is cs 2450 prerequisit?", role=Role.USER, is_init=False)

message = stream_response['choices'][0]['message']['content']
message

" CS 2450, which is a Computer Science course offered at Utah Valley University (UVU), typically has the following prerequisites:\n\n1. A strong background in programming concepts and principles using a high-level language such as C++ or Java. This usually means completing courses like CS 1310 or an equivalent before taking CS 2450.\n2. Familiarity with data structures, algorithms, and their complexities. Students are expected to have taken CS 2410 (Data Structures and Algorithms) or a similar course as a prerequisite for CS 2450.\n3. A solid foundation in discrete mathematics concepts like logic, set theory, relations, functions, propositional and predicate logic, and proof techniques. This is usually covered in courses such as MATH 1710 or an equivalent math course before taking CS 2450.\n\nIt's essential to consult the UVU catalog for the most up-to-date information on prerequisites since they may change from semester to semester. If you have any doubts, it is always a good idea to 

##  IngestAPI

In [16]:
import requests
from typing import Optional, Dict, Any
import os

class FileIngestionAPI(BaseAPI):
    """
    A specialized API client for handling file ingestions,
    inheriting from the BaseAPI for HTTP request management.
    """

    def __init__(self, base_url: str = "http://localhost:8000/v1"):
        super().__init__(base_url)

    def ingest_file(self, file_path: str, metadata: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
        """
        Ingest a file into the system with optional metadata.

        Args:
            file_path: The local path to the file to be ingested.
            metadata: Optional metadata to accompany the file during ingestion.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        endpoint = "ingest/file"
        # Prepare the file in the correct format for multipart/form-data
        files = {'file': (os.path.basename(file_path), open(file_path, 'rb'), 'multipart/form-data')}

        # Prepare additional data if metadata is provided
        data = {}
        if metadata is not None:
            # Assuming metadata needs to be sent as part of the data payload
            # Convert metadata dictionary to a series of ('key', 'value') tuples
            for key, value in metadata.items():
                data[key] = value

        response = self._post(endpoint, data=data, files=files)
        return response

    def list_ingested_documents(self) -> Optional[Dict[str, Any]]:
        """
        Retrieves a list of all ingested documents, including their IDs and metadata.

        Returns:
            Optional[Dict[str, Any]]: The JSON response containing the list of documents.
        """
        endpoint = "ingest/list"
        response = self._get(endpoint)
        return response

    def delete_ingested_document(self, doc_id: str) -> Optional[Dict[str, Any]]:
        """
        Deletes a specific ingested document by its ID.

        Args:
            doc_id: The ID of the document to delete.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of success or error.
        """
        endpoint = f"ingest/{doc_id}"
        response = self._delete(endpoint)
        return response

    def _delete(self, endpoint: str) -> Optional[Dict[str, Any]]:
        """
        Performs a DELETE request to a specified endpoint.

        Args:
            endpoint: The API endpoint to send the DELETE request to.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        url: str = f"{self.base_url}/{endpoint}"
        try:
            response = requests.delete(url, headers=self.headers)
            response.raise_for_status()
            # Assuming the DELETE endpoint might not always return a body/content
            if response.text:
                return response.json()
            return {"message": "Document deleted successfully"}
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
        return None


In [14]:
# Initialize the FileIngestionAPI with your service's base URL
file_ingestion_api = FileIngestionAPI(base_url='https://0e80-161-28-242-150.ngrok-free.app/v1')

Document ID: 3137127a-2bcb-4bd9-bd68-004fa6c42354, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: b01c1216-f3aa-4077-b0b4-61e771c68aec, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: 67ae272b-399f-443d-809c-6db7a121e6ea, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: be6cb7cd-37d8-4279-a993-5fdbce36dc03, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: 8ec2fea0-c5cd-45db-952a-58394e5659d5, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: 2b895cfa-7863-4dc0-94ab-b3b27d28f81e, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: 7b220b9b-94c9-4885-bdf5-1baaedbdf9dc, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: 0429490b-61ff-42ca-9e1c-83cf1f4e6b56, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document ID: aecc9276-0c00-4d1d-8c81-2583fc1b9b60, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Document I

#### Uploading file

In [ ]:
# Path to the file you wish to ingest
file_path = "/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu_update.md"

# Optional metadata, if any
metadata = {"author": "Group 2", "title": "Sample Document for UVU Course Catalog", "category": "Advising"}

# Ingest the file and receive the response
response = file_ingestion_api.ingest_file(file_path, metadata)

# Print the response
if response and 'data' in response:
    for document in response['data']:
        doc_id = document['doc_id']
        doc_metadata = document.get('doc_metadata', {})
        print(f"Document ID: {doc_id}, Metadata: {doc_metadata}")


#### List of files

In [18]:
file_ingestion_api = FileIngestionAPI(base_url='https://0e80-161-28-242-150.ngrok-free.app/v1')
documents = file_ingestion_api.list_ingested_documents()
if documents:
    for doc in documents.get('data', []):
        print(f"Doc ID: {doc['doc_id']}, Metadata: {doc.get('doc_metadata', {})}")


Doc ID: 3137127a-2bcb-4bd9-bd68-004fa6c42354, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: b01c1216-f3aa-4077-b0b4-61e771c68aec, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: 67ae272b-399f-443d-809c-6db7a121e6ea, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: be6cb7cd-37d8-4279-a993-5fdbce36dc03, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: 8ec2fea0-c5cd-45db-952a-58394e5659d5, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: 2b895cfa-7863-4dc0-94ab-b3b27d28f81e, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: 7b220b9b-94c9-4885-bdf5-1baaedbdf9dc, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: 0429490b-61ff-42ca-9e1c-83cf1f4e6b56, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: aecc9276-0c00-4d1d-8c81-2583fc1b9b60, Metadata: {'file_name': 'course_catelog_cs_uvu_update.md'}
Doc ID: 67011f22-4fbe-44f5-b842-906e9fe84cf4, Metadata:

#### Deleting Ingested document

In [19]:
doc_id = 'db6d5753-c444-48cc-8c62-1600e23d4558' # Valid document ID from the list
response = file_ingestion_api.delete_ingested_document(doc_id)
if response:
    print(response.get("message", "No message"))